In [12]:
import pickle
import os
import numpy as np
import torch
import torch.optim as optim
import datasets
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
from datasets import Forest, ToTensor
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights
from torchvision.models import ResNet50_Weights
from torch.autograd import Variable
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform = transforms.Compose([
    ToTensor(),
])

train_dataset = Forest(root_dir="/home/k45848/multispectral-imagery-segmentation/data/clean_data/train", label=True, transform=transform)
eval_dataset = Forest(root_dir="/home/k45848/multispectral-imagery-segmentation/data/clean_data/eval", label=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=10, shuffle=True)

for X, Y in train_loader:
    print(X.shape)
    print(Y.shape)
    break

torch.Size([10, 11, 64, 64])
torch.Size([10, 64, 64])


In [31]:
num_classes = 6
weights = DeepLabV3_ResNet50_Weights
weights_backbone = ResNet50_Weights.DEFAULT
model = deeplabv3_resnet50(weights=None, weights_backbone=weights_backbone, num_classes=num_classes)


In [20]:
b = model.backbone

In [ ]:
# Modify the model for 10 channels input and 3 classes output
#### input channels = 10, output classes = 3
input_channels = 11
model.backbone.conv1 = torch.nn.Conv2d(input_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.to(device)

In [103]:
criterion = nn.CrossEntropyLoss()
optimiser = optim.SGD(model.parameters(), lr=0.001)
num_epochs = 10


In [76]:
def compute_iou(preds, labels):
    iou = 0
    for cls in range(num_classes):
        intersection = ((preds== cls) & (labels==cls)).sum()
        union = ((preds== cls) | (labels==cls)).sum()
        if union > 0:
            iou += float(intersection) / float(union)
    return iou / num_classes

In [ ]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    intersection = 0
    union = 0
    total_iou = 0
    pbar = tqdm(total=len(train_loader), desc="[Train]")

    for images, labels  in train_loader:

        # reset gradients
        optimiser.zero_grad()

        # move data to gpu if model is on gpu
        images =images.cuda()
        labels = labels.cuda()

        # forward pass
        preds = model(images)["out"]
        loss = criterion(preds, labels)

        # compute accuracy
        predicted_labels = torch.argmax(preds, dim=1)
        intersection += torch.logical_and(predicted_labels, labels).sum().item()
        union += torch.logical_or(predicted_labels, labels).sum().item()

        # backward pass
        loss.backward()
        optimiser.step()

        

        train_loss += loss.item() * images.size(0)

    epoch_loss = train_loss / len(train_dataset)
    epoch_iou = intersection/union



Training

In [104]:
# Training loop with tqdm
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    
    # Wrap the train_loader with tqdm to add a progress bar
    with tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as t:
        for images, labels in t:
            # Move data to device (e.g., GPU if available)
            images = images.to(device)
            labels = labels.to(device)
            
            # Zero the parameter gradients
            optimiser.zero_grad()
            
            # Forward pass
            outputs = model(images)['out']
            
            # Calculate loss
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimiser.step()
            
            # Print statistics
            running_loss += loss.item() * images.size(0)
            t.set_postfix(loss=running_loss / ((t.n + 1) * train_loader.batch_size))
            t.update()
    
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

print('Training finished.')

/home/k45848/miniconda3/envs/Thesis/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712609048481/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Epoch 1/10: 100%|██████████| 1181/1181 [01:23<00:00, 14.18batch/s, loss=0.655]


Epoch [1/10], Loss: 0.6929


Epoch 2/10: 100%|██████████| 1181/1181 [01:19<00:00, 14.78batch/s, loss=0.46]


Epoch [2/10], Loss: 0.4873


Epoch 3/10: 100%|██████████| 1181/1181 [01:13<00:00, 15.98batch/s, loss=0.435]


...

Epoch [8/10], Loss: 0.3945


Epoch 9/10: 100%|██████████| 1181/1181 [01:22<00:00, 14.33batch/s, loss=0.368]


Epoch [9/10], Loss: 0.3901


Epoch 10/10: 100%|██████████| 1181/1181 [01:18<00:00, 14.99batch/s, loss=0.359]

Epoch [10/10], Loss: 0.3800
Training finished.
